In [14]:
import sys, pathlib
sys.path.insert(0, str(pathlib.Path.cwd().parent.parent))  # add repo root
from tsum import tsum
import torch
import json

from ndtools import fun_binary_graph as fbg # ndtools available at github.com/jieunbyun/network-datasets
from ndtools.graphs import build_graph
from pathlib import Path
import networkx as nx   


In [15]:
DATASET = Path("data") 

nodes = json.loads((DATASET / "nodes.json").read_text(encoding="utf-8"))
edges = json.loads((DATASET / "edges.json").read_text(encoding="utf-8"))
probs_dict = json.loads((DATASET / "probs.json").read_text(encoding="utf-8"))

# build base graph
G_base: nx.Graph = build_graph(nodes, edges, probs_dict)

# all edges ON (example); add node/edge 0s as needed
states = {eid: 1 for eid in edges.keys()}

k_val, status, _ = fbg.eval_global_conn_k(states, G_base, target_k=2)
print("k =", k_val, "status =", status)

k = 2 status = s


In [16]:
s_fun = lambda comps_st: fbg.eval_global_conn_k(comps_st, G_base, target_k=2)
row_names = list(edges.keys()) + ['sys']
n_state = 2  # binary states: 0, 1

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
probs = [[probs_dict[n]['0']['p'], probs_dict[n]['1']['p']] for n in row_names[:-1]]
probs = torch.tensor(probs, dtype=torch.float32, device=device)

In [19]:
# Two options for rule extraction: tsum.run_rule_extraction or tsum.run_rule_extraction_by_mcs

#result = tsum.run_rule_extraction( 
result = tsum.run_rule_extraction_by_mcs(
    # Problem-specific callables / data
    sfun=s_fun,
    probs=probs,
    row_names=row_names,
    n_state=n_state,
    output_dir="tsum_res",
    surv_json_name="rules_surv.json",
    fail_json_name="rules_fail.json"
) 

---
Round: 1, Unk. prob.: 1.000e+00
No. of non-dominant rules: 0, Survival rules: 0, Failure rules: 0
Failure sample found from sampling.
No. of existing rules removed:  0
New rule added. System state: f, System value: 2. Total samples: 1000000.
New rule (No. of conditions: 1): {'e03': ('<=', 0), 'sys': ('<=', 0)}
Updated sys_vals: [2]
---
Round: 2, Unk. prob.: 1.000e+00
No. of non-dominant rules: 1, Survival rules: 0, Failure rules: 1
Failure sample found from sampling.
No. of existing rules removed:  0
New rule added. System state: f, System value: 2. Total samples: 1000000.
New rule (No. of conditions: 1): {'e11': ('<=', 0), 'sys': ('<=', 0)}
---
Round: 3, Unk. prob.: 7.995e-01
No. of non-dominant rules: 2, Survival rules: 0, Failure rules: 2
Failure sample found from sampling.
No. of existing rules removed:  0
New rule added. System state: f, System value: 1. Total samples: 1000000.
New rule (No. of conditions: 1): {'e04': ('<=', 0), 'sys': ('<=', 0)}
Updated sys_vals: [1, 2]
---
R